In [1]:
import sys
sys.path.append("..")

from citibikeAPI import *
from utils import *


In [2]:
def findMostCommonRoutesByStation(trip_data):
	station_data = getAllStationData()
	#we only want trips that dont start and end at the same station
	trips = trip_data[trip_data['start station id'] != trip_data['end station id']].value_counts(['start station name','end station name'])
	#find unique starting stations
	unique_start_stations = list(set([trips.index[i][0] for i in range(len(trips))]))
	adjacencyDict = {}
	for station in unique_start_stations: 
		try: 
			id = station_data.index[station_data['name']==station][0]
			top_ten_routes = trips[station][:10]
			edges = []
			for route in top_ten_routes.index: 
				edges.append(station_data.index[station_data['name']==route][0])
			adjacencyDict[id] = edges
		#sometimes we get an index error if a station went online / offline since the last data pull 
		except IndexError: 
			pass 
	return adjacencyDict

In [4]:
d = getAllStationData()
trip_data = pd.read_csv('../../raw/202003-citibike-tripdata.csv')
adjacenyDict = findMostCommonRoutesByStation(trip_data)


In [5]:
d.head()
locData = d
locData = locData.reset_index()
locData.head()

,station_id,name,lat,lon,capacity,legacy_id,num_docks_available,num_bikes_disabled,num_ebikes_available,num_bikes_available,num_docks_disabled,station_status,is_renting,is_returning,last_reported,is_installed
0,69f84045-48f9-4c6c-90d3-0503e5ed39e9,Prospect Pl & Underhill Ave,40.676969,-73.965790,23,3574,19,3,1,1,0,active,1,1,1680615918,1
1,475c44e1-31e3-4c60-b071-af2ca3f7618c,Prospect Pl & Nostrand Ave,40.674280,-73.950300,22,4915,20,0,0,2,0,active,1,1,1680615918,1
2,66de11bc-0aca-11e7-82f6-3863bb44ef7c,Wolcott St & Dwight St,40.675327,-74.010070,21,3340,4,0,0,17,0,active,1,1,1680615919,1
3,66db9c2e-0aca-11e7-82f6-3863bb44ef7c,Avenue D & E 12 St,40.725806,-73.974225,41,339,37,0,0,4,0,active,1,1,1680615920,1
4,66de0cab-0aca-11e7-82f6-3863bb44ef7c,Columbia St & Lorraine St,40.674706,-74.007557,37,3333,3,0,0,34,0,active,1,1,1680615920,1


In [6]:
#min max lat/long for normalize
# need to filter out zeros for some data noise 
minlat = min([i for i in d['lat'] if i > 0])
maxlat  = max([i for i in d['lat'] if i > 0])
minlong = min([i for i in d['lon'] if i  < 0])
maxlong = max([i for i in d['lon'] if i < 0])

#add two columns and normalize
#hardcoding column locations to overwrite ""
d['x'] = 0
d['y'] = 0
d['adjacency'] = ''
locData = d
locData = locData.reset_index()
# print(adjacenyDict)
for r in range(d.shape[0]):
    d.iloc[r,-3] = clamp((d.iloc[r, 1] - minlat)/(maxlat-minlat),0,1) #lat
    d.iloc[r,-2] = clamp((d.iloc[r, 2] - minlong)/(maxlong-minlong),0,1) #lat
    try: 
        d.iloc[r, -1] = ' '.join([str(locData.index[locData['station_id']==v][0]) for v in adjacenyDict[locData.iloc[r,0]]])
    except KeyError: pass

locData = d[['x', 'y', 'adjacency']]


In [7]:
locData.head()

,x,y,adjacency
station_id,,,
69f84045-48f9-4c6c-90d3-0503e5ed39e9,0.175126,0.589088,
475c44e1-31e3-4c60-b071-af2ca3f7618c,0.164319,0.664557,
66de11bc-0aca-11e7-82f6-3863bb44ef7c,0.168528,0.373353,287 190 1454 406 217 753 737 337 1682 976
66db9c2e-0aca-11e7-82f6-3863bb44ef7c,0.371356,0.547992,
66de0cab-0aca-11e7-82f6-3863bb44ef7c,0.166029,0.385594,287 1575 1221 1378 14 991 1502 765 552 794


In [8]:
locData.to_csv('../../raw/locdata.csv')